In [ ]:
import scanpy as sc
import numpy as np
import pandas as pd
import os

# Preporcess

In [ ]:
def helper_read(filepath: str) -> sc.AnnData:
    adata = sc.read(os.path.join(filepath, "matrix.mtx"))
    adata = adata.T
    barcodes = pd.read_csv(os.path.join(filepath, "barcodes.tsv"), sep = '\t', header = None)
    features = pd.read_csv(os.path.join(filepath, "features.tsv"), sep = '\t', header = None)
    adata.obs_names = barcodes[0]
    adata.var_names = features[1]
    adata.var_names_make_unique()
    return adata

In [ ]:
filepath = "/media/duan/DuanLab_Data/LPS_10x/Cleandata/LPS03_cr/solote_outs/LPS03_SoloTE_output/LPS03_familytes_MATRIX"

In [ ]:
adata = helper_read(filepath)

In [ ]:
adata.write_h5ad("/media/duan/DuanLab_Data/LPS_10x/Cleandata/LPS03_cr/solote_outs/LPS03_SoloTE_output/LPS03_familytes_MATRIX.h5ad")

In [ ]:
filepath = "/media/duan/DuanLab_Data/LPS_10x/Cleandata/SA03_cr/solote_outs/SA03_SoloTE_output/SA03_familytes_MATRIX"

In [ ]:
adata = helper_read(filepath)

In [ ]:
adata.write_h5ad("/media/duan/DuanLab_Data/LPS_10x/Cleandata/SA03_cr/solote_outs/SA03_SoloTE_output/SA03_familytes_MATRIX.h5ad")

In [ ]:
adata1 = sc.read_h5ad("/media/duan/DuanLab_Data/LPS_10x/Cleandata/LPS03_cr/solote_outs/LPS03_SoloTE_output/LPS03_familytes_MATRIX.h5ad")
adata1.obs['Sample'] = 'LPS'
adata2 = sc.read_h5ad("/media/duan/DuanLab_Data/LPS_10x/Cleandata/SA03_cr/solote_outs/SA03_SoloTE_output/SA03_familytes_MATRIX.h5ad")
adata2.obs['Sample'] = 'Sham'
adata = sc.concat([adata1,adata2], merge = 'same', join = 'outer')
adata

In [ ]:
adata.obs_names_make_unique()

In [ ]:
sc.pp.filter_cells(adata, min_counts=100)
sc.pp.filter_cells(adata, min_genes=10)

In [ ]:
adata

In [ ]:
# mitochondrial genes, "MT-" for human, "Mt-" for mouse
adata.var["mt"] = adata.var_names.str.startswith("mt-")
# ribosomal genes
adata.var["ribo"] = adata.var_names.str.startswith(("Rps", "Rpl"))
# hemoglobin genes
adata.var["hb"] = adata.var_names.str.contains("^Hb[^(p)]")

adata.var["TE"] = adata.var_names.str.startswith(("SoloTE"))

In [ ]:
sc.pp.calculate_qc_metrics(
    adata, qc_vars=["mt", "ribo", "hb"], inplace=True, log1p=True
)

In [ ]:
sc.pl.violin(
    adata,
    ["n_genes_by_counts", "total_counts", "pct_counts_mt", "pct_counts_ribo","pct_counts_hb"],
    jitter=0.4,
    multi_panel=True,
)

In [ ]:
import numpy as np

In [ ]:
np.percentile(adata.obs["pct_counts_hb"], 98)

In [ ]:
np.percentile(adata.obs["pct_counts_ribo"], 98)

In [ ]:
np.percentile(adata.obs["total_counts"], 2)

In [ ]:
np.percentile(adata.obs["total_counts"], 98)

In [ ]:
np.percentile(adata.obs["n_genes_by_counts"], 98)

In [ ]:
np.percentile(adata.obs["n_genes_by_counts"], 2)

In [ ]:
adata = adata[adata.obs.pct_counts_mt < 20, :].copy()
adata = adata[adata.obs.pct_counts_hb < 5, :].copy()
adata = adata[adata.obs.pct_counts_ribo < 20, :].copy()
sc.pp.filter_cells(adata, min_counts=300)
sc.pp.filter_cells(adata, max_counts=40000)
sc.pp.filter_cells(adata, max_genes = 8000)
sc.pp.filter_cells(adata, min_genes=100)
adata

In [ ]:
# gene selection and normalization by Pearson residuals, and PCA on top
adata.layers['counts'] = adata.X.copy()
sc.experimental.pp.recipe_pearson_residuals(adata, batch_key = 'Sample', n_top_genes = 2000)

In [ ]:
sc.external.pp.harmony_integrate(adata, key = 'Sample', basis='X_pca', adjusted_basis='X_pca_harmony', max_iter_harmony = 100)

In [ ]:
sc.pl.embedding(adata, basis = 'X_pca_harmony', color = 'Sample')

In [ ]:
sc.pp.neighbors(adata, n_neighbors=15, use_rep='X_pca_harmony')
sc.tl.umap(adata)
sc.pl.umap(adata, color = 'Sample')

In [ ]:
adata.write_h5ad("LPS_TEfamily_Itg.h5ad")

In [ ]:
sc.tl.leiden(adata)

In [ ]:
adata.obsm['X_umap'][adata.obs['Sample'] == 'LPS',0] += 10

# Results

In [ ]:
sc.pl.umap(adata, color = 'leiden', legend_loc = 'on data')

In [ ]:
adata

In [ ]:
sum(adata.var['TE'])

In [ ]:
import cosg

In [ ]:
tedata = adata[:, adata.var['TE']].copy()

In [ ]:
for ct in tedata.obs['leiden'].unique():
    tmp = tedata[tedata.obs['leiden'] == ct].copy()
    if len(tmp.obs['Sample'].unique()) == 2:
        cosg.cosg(tmp, key_added='cosg', mu=1,expressed_pct=0.1,remove_lowly_expressed=True,
              n_genes_user=5, groupby='Sample')
        adata.uns[f'cosg_leiden_{ct}'] = tmp.uns['cosg']

In [ ]:
cosg.cosg(adata, key_added='cosg', mu=1,expressed_pct=0.1,remove_lowly_expressed=True,
              n_genes_user=10, groupby='leiden')

In [ ]:
adata.uns['cosg']

In [ ]:
sc.pl.umap(adata, color = ['Mrc1', 'P2ry12', 'Pecam1'], vmax = 'p99')

In [ ]:
# Macrophage
sc.pl.violin(adata[adata.obs['leiden'].isin(['24'])], keys=['SoloTE|ERV1', 'SoloTE|LTR'], groupby='Sample')

In [ ]:
# Microglia
sc.pl.violin(adata[adata.obs['leiden'].isin(['9'])], keys=['SoloTE|ERV1', 'SoloTE|LTR'], groupby='Sample')

In [ ]:
# Endothelial
sc.pl.violin(adata[adata.obs['leiden'].isin(['0'])], keys=['SoloTE|ERV1', 'SoloTE|LTR'], groupby='Sample')

In [ ]:
# 衰老里面和Rag1里面认为的神经元差异在这里几乎没有
sc.pl.violin(adata[adata.obs['leiden'].isin(['12', '28', '4'])], keys=['SoloTE|L2', 'SoloTE|L1', 'SoloTE|ERVK'], groupby='Sample')

In [ ]:
sc.pl.umap(adata, color = ['SoloTE|ERV1', 'SoloTE|LTR'], vmax = 'p99')

In [ ]:
import scanpy as sc
import cosg
import pandas as pd
import numpy as np

In [ ]:
adata = sc.read_h5ad("LPS_TEfamily_Itg.h5ad")
adata

In [ ]:
sc.tl.leiden(adata)

In [ ]:
sc.pl.umap(adata, color = 'leiden')

In [ ]:
adata.obsm['X_umap'][adata.obs['Sample'] == 'LPS',0] += 40

In [ ]:
sc.set_figure_params(figsize = (8,4), scanpy = False)
sc.pl.umap(adata, color = 'leiden', size = 15)

In [ ]:
cosg.cosg(adata, key_added='cosg', mu=1,expressed_pct=0.1,remove_lowly_expressed=True,
              n_genes_user=50, groupby='leiden')

In [ ]:
sc.pl.umap(adata, color = 'Camk2a', size = 15, vmax = 'p99')

In [ ]:
annotation_dict = {
    '0': 'Endothelial cells',
    '1': 'Astrocytes',
    '2': 'Immature neurons',
    '3': 'Astrocytes',
    '4': 'DG',
    '5': 'Astrocytes',
    '6': 'Astrocytes',
    '7': 'Immature neurons',
    '8': 'OPC',
    '9': 'Microglia',
    '10': 'Unknown',
    '11': 'Immature neurons',
    '12': 'CA1',
    '13': 'Pericytes',
    '14': 'Oligodendrocytes',
    '15': 'Tfap2c cells',
    '16': 'Oligodendrocytes',
    '17': 'Reln interneurons',
    '18': 'NSCs/NPCs',
    '19': 'Unknown',
    '20': 'Fibroblasts',
    '21': 'Oligodendrocytes',
    '22': 'Macrophage',
    '23': 'NSCs/NPCs',
    '24': 'Unknown',
    '25': 'Spock1 neurons',
    '26': 'Erythrocyte',
    '27': 'CHOR',
    '28': 'Gad2 interneurons',
    '29': 'Ependymal cells',
    '30': 'Unknown'
}

In [ ]:
with open("ACT.txt", "w") as handle:
    for i in df_tmp.index:
        line = "chuster_{}:".format(str(i))
        for g in df_tmp.loc[i,:]:
            line += "{},".format(g)
        handle.write(line[0:-1] + "\n")

In [ ]:
#sc.tl.dendrogram(adata,groupby=groupby,use_rep='X_pca')
df_tmp=pd.DataFrame(adata.uns['cosg']['names'][:20,]).T
#df_tmp=df_tmp.reindex(adata.uns['dendrogram_'+groupby]['categories_ordered'])
marker_genes_list={idx: list(row.values) for idx, row in df_tmp.iterrows()}
marker_genes_list = {k: v for k, v in marker_genes_list.items() if not any(isinstance(x, float) for x in v)}

sc.pl.dotplot(adata, marker_genes_list,
             groupby="leiden",
             dendrogram=False,
             swap_axes=True,
             standard_scale='var',
             cmap='RdYlBu_r', colorbar_title='column scaled\nexpression', var_group_rotation = 0,
             mean_only_expressed = True, linewidth = 0, marker = "o")

In [ ]:
adata.obs['cell_type'] = [annotation_dict[x] for x in adata.obs['leiden']]

In [ ]:
adata = adata[adata.obs['cell_type']!='Unknown'].copy()

In [ ]:
sc.pl.umap(adata, color = 'cell_type')

In [ ]:
adata.obsm['X_umap'][adata.obs['Sample'] == 'LPS',0] -= 40

In [ ]:
sc.set_figure_params(figsize=(6,6), scanpy = False, dpi_save=600)
sc.pl.umap(adata, color = 'cell_type', frameon = False, size = 30, save = "celltype.pdf")

In [ ]:
sc.set_figure_params(figsize=(6,6), scanpy = False, dpi_save=600)
sc.pl.umap(adata, color = 'Sample', frameon = False, size = 30, save = "Batch.pdf", palette=['#0000FF', '#FF0000'])

In [ ]:
adata.obsm['X_umap'][adata.obs['Sample'] == 'LPS',0] += 40

In [ ]:
sc.set_figure_params(figsize=(12,6), scanpy = False, dpi_save=600)
sc.pl.umap(adata, color = ['SoloTE|RLTR4_Mm', 'SoloTE|MMVL30-int'], frameon = False, size = 30, vmax = 'p99')

In [ ]:
adata

In [ ]:
tedata = adata[:, adata.var['TE']].copy()

In [ ]:
for ct in tedata.obs['leiden'].unique():
    tmp = tedata[tedata.obs['leiden'] == ct].copy()
    if len(tmp.obs['Sample'].unique()) == 2:
        cosg.cosg(tmp, key_added='cosg', mu=1,expressed_pct=0.1,remove_lowly_expressed=True,
              n_genes_user=10, groupby='Sample')
        adata.uns[f'cosg_{ct}'] = tmp.uns['cosg']

In [ ]:
adata.uns['cosg_30']

In [ ]:
sc.pl.umap(adata, color = ['SoloTE|ERVL-MaLR'], frameon = False, size = 30, vmax = 'p99')

In [ ]:
sc.pl.umap(adata, color = ['SoloTE|LTR'], frameon = False, size = 30, vmax = 'p99')

In [ ]:
sc.pl.umap(adata, color = ['SoloTE|ERV1'], frameon = False, size = 30, vmax = 'p99')

In [ ]:
TE_df = pd.read_table("/media/duan/sda2/Reference_DataBase/refdata-gex-GRCm39-2024-A/genes/mm39_rmsk.txt.gz")

In [ ]:
TE_df[TE_df['repName'] == 'MTEa']

In [ ]:
adata.uns['cosg_Endothelial cells']

In [ ]:
sc.pl.umap(adata, color = ['SoloTE|RLTR4_Mm', 'SoloTE|L1M3',
                          'SoloTE|MTEa', 'SoloTE|RLTR31C_MM',
                          'SoloTE|MER20B', 'SoloTE|RLTR4_MM-int',
                          'SoloTE|IAPEz-int', 'SoloTE|MMVL30-int',
                          'SoloTE|L1MB4'], frameon = False, size = 30, vmax = 'p99', ncols = 2, save = 'EC_DETE.pdf')

In [ ]:
marker_genes_list = {
    'Endothelial cells': ['Flt1', 'Cldn5'],
    'Astrocytes': ['Slc1a3', 'Aqp4'],
    'Immature neurons': ['Dcx', 'Neurod1'],
    'DG': ['C1ql3', 'C1ql2'],
    'OPC': ['Cspg4', 'Pdgfra'],
    'Microglia': ['Ctss', 'Csf1r'],
    'CA1': ['Fibcd1', 'Prkcg'],
    'Pericytes': ['Rgs5', 'Notch3'],
    'Oligodendrocytes': ['Mog', 'Ermn'],
    'Tfap2c cells': ['Tfap2c', 'Gas1'],
    'Reln interneurons': ['Reln', 'Ndnf'],
    'NSCs/NPCs': ['Top2a', 'Cdca8'],
    'Fibroblasts': ['Col1a1', 'Dcn'],
    'Macrophage': ['Pf4', 'Mrc1'],
    'Spock1 neurons': ['Spock1', 'Cpne4'],
    'Erythrocyte': ['Hba-a1', 'Hbb-bt'],
    'CHOR': ['Prlr', 'Ttr'],
    'Gad2 interneurons': ['Gad2', 'Slc32a1'],
    'Ependymal cells': ['Ccdc153', 'Tmem212']
}

In [ ]:
marker_genes_list = {
     'Astrocytes': ['Slc1a3', 'Aqp4'],'CA1': ['Fibcd1', 'Prkcg'], 'CHOR': ['Prlr', 'Ttr'], 'DG': ['C1ql3', 'C1ql2'],
     'Endothelial cells': ['Flt1', 'Cldn5'], 'Ependymal cells': ['Ccdc153', 'Tmem212'], 'Erythrocyte': ['Hba-a1', 'Hbb-bt'],
    'Fibroblasts': ['Col1a1', 'Dcn'],'Gad2 interneurons': ['Gad2', 'Slc32a1'], 'Immature neurons': ['Dcx', 'Neurod1'],
     'Macrophage': ['Pf4', 'Mrc1'],'Microglia': ['Ctss', 'Csf1r'],'NSCs/NPCs': ['Top2a', 'Cdca8'], 'OPC': ['Cspg4', 'Pdgfra'],
     'Oligodendrocytes': ['Mog', 'Ermn'],'Pericytes': ['Rgs5', 'Notch3'], 'Reln interneurons': ['Reln', 'Ndnf'],  'Spock1 neurons': ['Spock1', 'Cpne4'],
    'Tfap2c cells': ['Tfap2c', 'Gas1'],
}

In [ ]:
sc.pl.dotplot(adata, marker_genes_list,
             groupby="cell_type",
             dendrogram=False,
             swap_axes=False,
             standard_scale='var',
             cmap='RdYlBu_r', colorbar_title='column scaled\nexpression', var_group_rotation = 90,
             mean_only_expressed = True, linewidth = 0, marker = "o", save = 'cell_type.pdf')

In [ ]:
adata.write_h5ad("LPS_TEsubfamily_Itg_anno.h5ad")

In [ ]:
adata = sc.read_h5ad("LPS_TEsubfamily_Itg_anno.h5ad")

In [ ]:
sc.pl.umap(adata, color = 'SoloTE|RLTR4_Mm', vmax = 'p99')

In [ ]:
sc.pl.umap(adata, color = 'SoloTE|RLTR4_MM-int', vmax = 'p99')

In [ ]:
import scanpy as sc

In [ ]:
adata = sc.read_h5ad("LPS_TEsubfamily_Itg_anno.h5ad")
adata

In [ ]:
del adata.varm
del adata.layers
del adata.obsp

In [ ]:
adata

In [ ]:
import numpy as np
from tqdm import tqdm
from scipy.sparse import csr_matrix

In [ ]:
tmp = adata.X.toarray()
tmp_q = np.percentile(tmp, 99, axis = 0)
for i in tqdm(range(len(tmp_q))):
    tmp[tmp[:,i] >= tmp_q[i], i] = tmp_q[i]

In [ ]:
adata.X = csr_matrix(tmp)

In [ ]:
sc.pl.umap(adata, color = 'SoloTE|MMVL30-int')

In [ ]:
del adata.uns['pearson_residuals_normalization']

In [ ]:
adata.write_h5ad("LPS_TEsubfamily_Itg_cirro.h5ad")

In [ ]:
adata

## Only use gene for clustering

In [ ]:
adata.obs_names_make_unique()
adata.var["TE"] = adata.var_names.str.startswith(("SoloTE"))

BACKUP = adata.copy()
adata = adata[:, ~adata.var['TE']].copy()

In [ ]:
adata.var["mt"] = adata.var_names.str.startswith("mt-")
# ribosomal genes
adata.var["ribo"] = adata.var_names.str.startswith(("Rps", "Rpl"))
# hemoglobin genes
adata.var["hb"] = adata.var_names.str.contains("^Hb[^(p)]")

sc.pp.calculate_qc_metrics(
    adata, qc_vars=["mt", "ribo", "hb"], inplace=True, log1p=True
)

In [ ]:
sc.pl.violin(
    adata,
    ["n_genes_by_counts", "total_counts", "pct_counts_mt", "pct_counts_ribo","pct_counts_hb"],
    jitter=0.4,
    multi_panel=True,
)

In [ ]:
adata = adata[adata.obs.pct_counts_mt < 20, :].copy()
adata = adata[adata.obs.pct_counts_hb < 5, :].copy()
adata = adata[adata.obs.pct_counts_ribo < 20, :].copy()
sc.pp.filter_cells(adata, min_counts=300)
sc.pp.filter_cells(adata, max_counts=60000)
sc.pp.filter_cells(adata, max_genes = 8000)
sc.pp.filter_cells(adata, min_genes=100)
adata

In [ ]:
# gene selection and normalization by Pearson residuals, and PCA on top
adata.layers['counts'] = adata.X.copy()
sc.experimental.pp.recipe_pearson_residuals(adata, batch_key = 'Sample', n_top_genes = 2000)

In [ ]:
sc.external.pp.harmony_integrate(adata, key = 'Sample', basis='X_pca', adjusted_basis='X_pca_harmony', max_iter_harmony = 100)

In [ ]:
sc.pl.embedding(adata, basis = 'X_pca_harmony', color = 'Sample')

In [ ]:
sc.pp.neighbors(adata, n_neighbors=15, use_rep='X_pca_harmony')
sc.tl.umap(adata)
sc.pl.umap(adata, color = 'Sample')

In [ ]:
sc.tl.leiden(adata)

In [ ]:
sc.pl.umap(adata, color = ['Ptprb', 'leiden'])

In [ ]:
adata.obsm['X_umap'][adata.obs['Sample'] == 'LPS',0] += 10
sc.pl.umap(adata, color = ['Sample'])

## 然后把TE合并回来，这一次只用TE，且只保留前2000 highly variable

In [ ]:
BACKUP[adata.obs_names, ]

In [ ]:
adata

In [ ]:
new_adata = BACKUP[adata.obs_names, :].copy()
new_adata.obs = adata.obs
new_adata.obsm = adata.obsm

In [ ]:
adata = new_adata.copy()

In [ ]:
from scipy.sparse import csr_matrix
adata.layers['counts'] = adata.X.copy()

In [ ]:
adata = adata[:, adata.var['TE']].copy()

In [ ]:
sc.experimental.pp.highly_variable_genes(adata, n_top_genes = 2000, batch_key='Sample')

In [ ]:
adata = adata[:, adata.var['highly_variable']].copy()

In [ ]:
# Freeman-Tukey (FT) transform
adata.X = csr_matrix(np.sqrt(adata.X.toarray()) + np.sqrt(adata.X.toarray() + 1))

## 进行差异分析

In [ ]:
sc.set_figure_params(figsize=(10,5), scanpy = False)
sc.pl.umap(adata, color = 'leiden', size = 20)

In [ ]:
import cosg

In [ ]:
for ct in adata.obs['leiden'].unique():
    tmp = adata[adata.obs['leiden'] == ct].copy()
    if len(tmp.obs['Sample'].unique()) == 2:
        cosg.cosg(tmp, key_added='cosg', mu=1,expressed_pct=0.1,remove_lowly_expressed=True,
              n_genes_user=50, groupby='Sample')
        adata.uns[f'cosg_leiden_{ct}'] = tmp.uns['cosg']

### Endothelial

In [ ]:
sc.pl.umap(adata, color = 'SoloTE|chr5|105249814|105250057|MTEa:ERVL-MaLR:LTR|27.3|+', size = 20)

In [ ]:
sc.pl.umap(adata, color = 'SoloTE|chr5|105249273|105249812|L1M3:L1:LINE|30.0|+', size = 20)

In [ ]:
sc.pl.umap(adata, color = 'SoloTE|chr5|105250070|105250149|RLTR31C_MM:ERVK:LTR|10.3|+', size = 20)

In [ ]:
sc.pl.umap(adata, color = 'SoloTE|chr18|60516527|60517104|Lx8:L1:LINE|26.9|-', size = 20)

In [ ]:
# 0: Endo
# 8: Microglia
# 23: Macrophage
# 22: Fibroblasts

### Microglia

In [ ]:
sc.pl.umap(adata, color = 'SoloTE|chr13|55334594|55335474|RLTR4_MM-int:LTR:LTR|31.6|+', size = 20, vmax = 'p99.9')

In [ ]:
sc.pl.umap(adata, color = 'SoloTE|chr11|83555632|83555906|L1MdV_I:L1:LINE|11.0|+', size = 20, vmax = 'p99.9')

In [ ]:
sc.pl.umap(adata, color = 'SoloTE|chr2|90278726|90279258|L1MdF_IV:L1:LINE|6.5|+', size = 20, vmax = 'p99.9')

In [ ]:
sc.pl.umap(adata, color = 'SoloTE|chr13|55335474|55335696|MMVL30-int:ERV1:LTR|.9|+', size = 20, vmax = 'p99.9')

In [ ]:
sc.pl.umap(adata, color = 'SoloTE|chr6|31779939|31781206|IAPEz-int:ERVK:LTR|8.9|-', size = 20, vmax = 'p99.9')

### Macrophage

In [ ]:
sc.pl.umap(adata, color = 'SoloTE|chr13|55334594|55335474|RLTR4_MM-int:LTR:LTR|31.6|+', size = 20, vmax = 'p99.9')

In [ ]:
sc.pl.umap(adata, color = 'SoloTE|chr2|75472715|75473289|RLTR6_Mm:ERV1:LTR|3.9|+', size = 20, vmax = 'p99.9')

In [ ]:
sc.pl.umap(adata, color = 'SoloTE|chr2|75470991|75471717|RLTR4_MM-int:LTR:LTR|34.5|+', size = 20, vmax = 'p99.9')

In [ ]:
sc.pl.umap(adata, color = 'SoloTE|chr11|83555632|83555906|L1MdV_I:L1:LINE|11.0|+', size = 20, vmax = 'p99.9')

In [ ]:
sc.pl.umap(adata, color = 'SoloTE|chr2|75471717|75471888|MMVL30-int:ERV1:LTR|1.8|+', size = 20, vmax = 'p99.9')

In [ ]:
sc.pl.umap(adata, color = 'SoloTE|chr13|55335474|55335696|MMVL30-int:ERV1:LTR|.9|+', size = 20, vmax = 'p99.9')

In [ ]:
sc.pl.umap(adata, color = 'SoloTE|chr2|75469015|75470605|MMVL30-int:ERV1:LTR|4.1|+', size = 20, vmax = 'p99.9')

In [ ]:
sc.pl.umap(adata, color = 'SoloTE|chr2|75471888|75472595|RLTR4_MM-int:LTR:LTR|33.2|+', size = 20, vmax = 'p99.9')

In [ ]:
sc.pl.umap(adata, color = 'SoloTE|chr2|164266546|164266658|MER3:hAT-Charlie:DNA|30.4|+', size = 20, vmax = 'p99.9')

### Fibroblast

In [ ]:
sc.pl.umap(adata, color = 'SoloTE|chr2|75472715|75473289|RLTR6_Mm:ERV1:LTR|3.9|+', size = 20, vmax = 'p99.9')

In [ ]:
sc.pl.umap(adata, color = 'SoloTE|chr2|75469015|75470605|MMVL30-int:ERV1:LTR|4.1|+', size = 20, vmax = 'p99.9')

In [ ]:
sc.pl.umap(adata, color = 'SoloTE|chr2|75471717|75471888|MMVL30-int:ERV1:LTR|1.8|+', size = 20, vmax = 'p99.9')

In [ ]:
sc.pl.umap(adata, color = 'SoloTE|chr2|75470991|75471717|RLTR4_MM-int:LTR:LTR|34.5|+', size = 20, vmax = 'p99.9')

In [ ]:
sc.pl.umap(adata, color = 'SoloTE|chr5|142990803|142991508|IAPEz-int:ERVK:LTR|12.9|+', size = 20, vmax = 'p99.9')

In [ ]:
sc.pl.umap(adata, color = 'SoloTE|chr10|4258075|4258217|B1_Mus1:Alu:SINE|7.0|-', size = 20, vmax = 'p99.9')

In [ ]:
sc.pl.umap(adata, color = 'SoloTE|chr10|4270793|4270950|RSINE1:B4:SINE|32.7|-', size = 20, vmax = 'p99.9')

In [ ]:
new_adata.write_h5ad('LPS_TElocus_annotated_raw.h5ad')